In [167]:
!pip install xlrd

In [1]:

import pandas as pd
def import_data(url):    
    shark_df = pd.read_excel("https://www.sharkattackfile.net/spreadsheets/GSAF5.xls")
    shark_df.dropna(axis=1, how='all', inplace=True)
    shark_df.dropna(axis=0, how='all', inplace=True)
    return shark_df

shark_df = import_data("https://www.sharkattackfile.net/spreadsheets/GSAF5.xls")

In [5]:
shark_df.columns

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Unnamed: 11', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [9]:
#Vasco
def clean_dates(df):
    new_df = df.copy()
    new_df['Date'] = pd.to_datetime(new_df['Date'], errors='coerce')

    #drop rows invalid dates
    new_df.dropna(subset=['Date'], inplace=True)

    #filter out dates before 1900
    new_df = new_df[new_df['Date'] >= pd.Timestamp('1900-01-01')].reset_index(drop=True)
    return new_df

<DatetimeArray>
[          '2024-10-14 00:00:00',           '2024-10-11 00:00:00',
           '2024-09-17 00:00:00',           '2024-09-16 00:00:00',
           '2024-08-26 00:00:00',           '2024-08-06 00:00:00',
           '2024-07-23 00:00:00',           '2024-07-18 00:00:00',
           '2024-07-08 00:00:00',           '2024-07-05 00:00:00',
 ...
 '1970-01-01 00:00:00.000001871', '1970-01-01 00:00:00.000001865',
           '1952-07-28 00:00:00', '1970-01-01 00:00:00.000001852',
 '1970-01-01 00:00:00.000001844', '1970-01-01 00:00:00.000001764',
 '1970-01-01 00:00:00.000001758', '1970-01-01 00:00:00.000001749',
 '1970-01-01 00:00:00.000001595', '1970-01-01 00:00:00.000001555']
Length: 4836, dtype: datetime64[ns]

In [6]:
def clean_cols(df):
    new_df = df.rename(columns={'Unnamed: 11': 'Fatal'})
    new_df = new_df.drop(['pdf', 'href formula', 'href','Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'], axis=1)
    new_df = new_df.drop_duplicates()
    return new_df

clean_df = clean_cols(shark_df).columns

In [9]:
shark_df["Country"].unique()

array(['Honduras', 'USA', 'Morocco', 'Jamaica', 'Belize', 'Australia',
       'Maldive Islands', 'Turks and Caicos', 'French Polynesia',
       'Tobago', 'Bahamas', 'AUSTRALIA', 'INDIA', 'TRINIDAD', 'BAHAMAS',
       'SOUTH AFRICA', 'MEXICO', 'NEW ZEALAND', 'EGYPT', 'Mexico',
       'BELIZE', 'PHILIPPINES', 'Coral Sea', 'SPAIN', 'PORTUGAL', 'SAMOA',
       'COLOMBIA', 'ECUADOR', 'FRENCH POLYNESIA', 'NEW CALEDONIA',
       'TURKS and CaICOS', 'CUBA', 'BRAZIL', 'SEYCHELLES', 'ARGENTINA',
       'FIJI', 'MeXICO', 'Maldives', 'South Africa', 'ENGLAND', 'JAPAN',
       'INDONESIA', 'JAMAICA', 'MALDIVES', 'THAILAND', 'COLUMBIA',
       'COSTA RICA', 'New Zealand', 'British Overseas Territory',
       'CANADA', 'JORDAN', 'ST KITTS / NEVIS', 'ST MARTIN',
       'PAPUA NEW GUINEA', 'REUNION ISLAND', 'ISRAEL', 'CHINA', 'IRELAND',
       'ITALY', 'MALAYSIA', 'LIBYA', nan, 'MAURITIUS', 'SOLOMON ISLANDS',
       'ST HELENA, British overseas territory', 'COMOROS', 'REUNION',
       'UNITED KINGDOM',

In [38]:
def clean_country(df):
    """Tidies the "Country" column of the DataFrama """
    new_df = df.copy()
    new_df = new_df.dropna(subset=["Country"])

    # Converts country column to consistent capitalisation and strips spaces
    new_df["Country"] = new_df["Country"].apply(lambda x: x.strip().title())
    new_df["Country"] = new_df["Country"].apply(lambda x: "USA" if x == "Usa" else x)

    #Removes rows that contain Oceans and Seas for the country
    new_df = new_df[~new_df["Country"].str.contains("Ocean", na=False)]
    new_df = new_df[~new_df["Country"].str.contains("Central Pacific", na=False)]
    new_df = new_df[~new_df["Country"].str.contains(" Sea", na=False)]
    new_df = new_df[~new_df["Country"].str.contains("Persian Gulf", na=False)]
    
    #Corrects country names
    new_df["Country"] = new_df["Country"].replace("Ceylon (Sri Lanka)", "Sri Lanka")
    new_df["Country"] = new_df["Country"].replace("Ceylon", "Sri Lanka")
    new_df["Country"] = new_df["Country"].replace("Maldive Islands", "Maldives")
    new_df["Country"] = new_df["Country"].replace("St. Maartin", "St Martin")
    new_df["Country"] = new_df["Country"].replace("St. Martin", "St Martin")
    new_df["Country"] = new_df["Country"].replace("Reunion Island", "Reunion")
    new_df["Country"] = new_df["Country"].replace("Trinidad", "Trinidad & Tobago")
    new_df["Country"] = new_df["Country"].replace("Tobago", "Trinidad & Tobago")
    new_df["Country"] = new_df["Country"].replace("Turks And Caicos", "Turks & Caicos")
    new_df["Country"] = new_df["Country"].replace("Sudan?", "Sudan")
    new_df["Country"] = new_df["Country"].replace("United Arab Emirates (Uae)?", "United Arab Emirates")
    new_df["Country"] = new_df["Country"].replace("United Arab Emirates (Uae)", "United Arab Emirates")
    new_df["Country"] = new_df["Country"].replace("Western Samoa", "Samoa")
    new_df["Country"] = new_df["Country"].replace("Scotland", "United Kingdom")
    new_df["Country"] = new_df["Country"].replace("Crete", "Greece")
    new_df["Country"] = new_df["Country"].replace("Okinawa", "Japan")
    new_df["Country"] = new_df["Country"].replace("Columbia", "Colombia")
    new_df["Country"] = new_df["Country"].replace("England", "United Kingdom")
    new_df["Country"] = new_df["Country"].replace("New Britain", "Papua New Guinea")
    new_df["Country"] = new_df["Country"].replace("New Guinea", "Papua New Guinea")
    new_df["Country"] = new_df["Country"].replace('St Helena, British Overseas Territory', "St Helena")
    new_df["Country"] = new_df["Country"].replace('Burma', "Myanmar")    

    #Counts occurences of each country
    country_counts = new_df["Country"].value_counts() #Contains 6923


    # Filter countries that appear more than two times
    countries_to_keep = country_counts[country_counts > 1].index
    new_df = new_df[new_df["Country"].isin(countries_to_keep)]   

    return new_df

print(sorted(list(clean_country(shark_df)["Country"].unique())))


['American Samoa', 'Antigua', 'Argentina', 'Australia', 'Azores', 'Bahamas', 'Barbados', 'Belize', 'Bermuda', 'Brazil', 'Canada', 'Cape Verde', 'Cayman Islands', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Fiji', 'France', 'French Polynesia', 'Greece', 'Grenada', 'Guam', 'Guinea', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Johnston Island', 'Kenya', 'Kiribati', 'Lebanon', 'Liberia', 'Libya', 'Madagascar', 'Malaysia', 'Maldives', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritius', 'Mexico', 'Micronesia', 'Montenegro', 'Mozambique', 'Myanmar', 'Namibia', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Nigeria', 'Norway', 'Palau', 'Panama', 'Papua New Guinea', 'Peru', 'Philippines', 'Portugal', 'Reunion', 'Russia', 'Samoa', 'Saudi Arabia', 'Senegal', 'Seychelles', 'Sierra Leone', 'Singapore', 'Solomon Islands', 

In [52]:
def hemisphere(df):
    new_df = df.copy()

    hemi_dict = {
        'American Samoa': "South", 
        'Antigua': 'North', 
        'Argentina': "South", 
        'Australia': "South", 
        'Azores': "North", 
        'Bahamas': "North", 
        'Barbados': "North", 
        'Belize': "North", 
        'Bermuda': "North", 
        'Brazil': "Equator", 
        'Myanmar': "North", 
        'Canada': "North", 
        'Cape Verde': "North", 
        'Cayman Islands': "North", 
        'Chile': "South", 
        'China': "North", 
        'Colombia': "Equator", 
        'Costa Rica': "North",
        'Croatia': "North", 
        'Cuba': "North", 
        'Dominican Republic': "North", 
        'Ecuador': "Equator", 
        'Egypt': "North",
        'El Salvador': "North", 
        'Fiji': "South", 
        'France': "North", 
        'French Polynesia': "South", 
        'Greece': "North",
        'Grenada': "North", 
        'Guam': "North", 
        'Guinea': "North", 
        'Guyana': "North", 
        'Haiti': "North", 
        'Honduras': "North", 
        'Hong Kong': "North", 
        'Iceland': "North",
        'India': "North", 
        'Indonesia': "Equator", 
        'Iran': "North", 
        'Iraq': "North", 
        'Ireland': "North",
        'Israel': "North", 
        'Italy': "North", 
        'Jamaica': "North", 
        'Japan': "North",
        'Johnston Island': "North", 
        'Kenya': "Equator", 
        'Kiribati': "Equator", 
        'Lebanon': "North", 
        'Liberia': "North", 
        'Libya': "North", 
        'Madagascar': "South",
        'Malaysia': "North", 
        'Maldives': "Equator", 
        'Malta': "North", 
        'Marshall Islands': "North", 
        'Martinique': "North", 
        'Mauritius': "South", 
        'Mexico': "North", 
        'Micronesia': "North", 
        'Montenegro': "North", 
        'Mozambique': "South", 
        'Namibia': "South", 
        'New Caledonia': "South", 
        'New Zealand': "South", 
        'Nicaragua': "North", 
        'Nigeria': "North", 
        'Norway': "North", 
        'Palau': "North", 
        'Panama': "North",
        'Papua New Guinea': "South", 
        'Peru': "South", 
        'Philippines': "North", 
        'Portugal': "North", 
        'Reunion': "South", 
        'Russia': "North",
        'Samoa': "South", 
        'Saudi Arabia': "North", 
        'Senegal': "North", 
        'Seychelles': "South", 
        'Sierra Leone': "North", 
        'Singapore': "North",
        'Solomon Islands': "South", 
        'Somalia': "Equator", 
        'South Africa': "South", 
        'South Korea': "North", 
        'Spain': "North", 
        'Sri Lanka': "South", 
        'St Helena, British Overseas Territory': "South", 
        'St Martin': "North",
        'St Helena': "South",
        'Sudan': "North", 
        'Taiwan': "North", 
        'Tanzania': "Equator",
        'Thailand': "North", 
        'Tonga': "South", 
        'Trinidad & Tobago': "North", 
        'Tunisia': "North", 
        'Turkey': "North", 
        'Turks & Caicos': "North",
        'USA': "North", 
        'United Arab Emirates': "North", 
        'United Kingdom': "North", 
        'Uruguay': "South", 
        'Vanuatu': "South",
        'Venezuela': "North", 
        'Vietnam': "North", 
        'West Indies': "North", 
        'Yemen': "North"
    }
    
    new_df["Hemisphere"] = new_df["Country"].apply(lambda country: hemi_dict.get(country, "Na"))
    
    #Code here
    return new_df

hemisphere(clean_country(shark_df))[["Hemisphere","Country"]]

,Hemisphere,Country
0,North,Honduras
1,North,USA
2,North,Honduras
4,North,Jamaica
5,North,Belize
...,...,...
6968,South,Australia
6969,South,Australia
6970,North,USA
6971,North,Panama


In [ ]:
shark_df["Unnamed: 11"].unique()

In [ ]:
shark_df["Unnamed: 11"].unique()

In [ ]:
shark_df["Unnamed: 21"].unique()

In [ ]:
shark_df["Unnamed: 22"].unique()

In [ ]:
shark_df.info()

In [ ]:
shark_df.nunique()

In [ ]:
print(shark_df.isna().sum())


In [ ]:
shark_df["Year"].unique()

In [ ]:
shark_df["Sex"].unique()

In [ ]:
shark_df.sort_values(by=['Year'], ascending=[True]).head(20)